In [1]:
import os
os.chdir('..')

import json
import openai
import logging

from answerbot.get_wikipedia import WikipediaApi
from answerbot.prompt_builder import FunctionalPrompt, Assistant, FunctionCall, FunctionResult, System
from answerbot.react_prompt import FunctionalReactPrompt, NewFunctionalReactPrompt, TextReactPrompt, Question
from answerbot.toolbox import WikipediaSearch
from answerbot.react import LLMReactor 

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

with open("config.json", "r") as f:
    json_config = json.load(f)
openai.api_key = json_config["api_key"]


system_prompt = "please answer the following question. you can use wikipedia for reference - but think carefully about what pages exist at wikipedia."
system_prompt = """Please answer the following question. You can use wikipedia for reference - but think carefully about what pages exist at wikipedia.
When you look for a property of something or someone - search for that something page instead of using that property in the search.
When you receive information from wikipedia check what useful informatiou have you found and what else do you need.
When you know the answer call finish. Please make the answer as short as possible. If it can be answered with yes or no that is best.
Remove all explanations from the answer and put them into the thought field.
The search function automatically retrieves the first search result and presents a short fragment from it. It is formatted in Markdown.
"""
#The pages are formated in Markdown - when looking up a section of an article lookup '##Section Name', when lookup up a word - just lookup 'word'.


#question = "What year did Guns N Roses perform a promo for a movie starring Arnold Schwarzenegger as a former New York Police detective?"
question = "What is the weight proportion of oxygen in water?"
#question = "The arena where the Lewiston Maineiacs played their home games can seat how many people?"
#question = "What is the name of the fight song of the university whose main campus is in Lawrence, Kansas and whose branch campuses are in the Kansas City metropolitan area?"
#question = "When Poland became elective monarchy?"

prompt = FunctionalPrompt([System(system_prompt), Question(question)])
summarize_prompt = System("Reflect on the received information and plan next steps.")
  
wiki_api = WikipediaApi(max_retries=3, chunk_size=400)
toolbox = WikipediaSearch(wiki_api)
model = 'gpt-4-1106-preview'

reactor = LLMReactor(model, toolbox, prompt, summarize_prompt, 7)


In [3]:
print(f">>>LLM call number: {reactor.step}")
reactor.process_prompt()

>>>LLM call number: 1


INFO:answerbot.react:FunctionCall('get', 'To find the molecular weight of water and atomic weights of hydrogen and oxygen to calculate the weight proportion of oxygen in water.', title='Properties_of_water')
INFO:answerbot.react:FunctionResult('get', 'Successfully retrieved \'Properties_of_water\' from Wikipedia.\nThe retrieved page contains the following sections:\n## Physical properties\n### Water, ice, and vapor\n#### Heat capacity and heats of vaporization and fusion\n#### Density of water and ice\n#### Density of saltwater and ice\n#### Miscibility and condensation\n#### Vapor pressure\n#### Compressibility\n#### Triple point\n#### Melting point\n### Electrical properties\n#### Electrical conductivity\n### Polarity and hydrogen bonding\n#### Cohesion and adhesion\n#### Surface tension\n#### Capillary action\n#### Water as a solvent\n#### Quantum tunneling\n### Electromagnetic absorption\n## Structure\n### Molecular structure\n## Chemical properties\n### Self-ionization\n### Geoche

In [4]:
print(f">>>LLM call number: {reactor.step}")
reactor.process_prompt()

>>>LLM call number: 2


INFO:answerbot.react:FunctionCall('lookup', 'To find the atomic weights of hydrogen and oxygen and the molecular weight of water to calculate the weight proportion of oxygen in water.', keyword='molecular structure')
INFO:answerbot.react:FunctionResult('lookup', 'Keyword "molecular structure" found  in: \n### Molecular structure\n\nSee also: [Molecular orbital diagram § Water](/wiki/Molecular_orbital_diagram#Water "Molecular orbital diagram")\n\nThe repulsive effects of the two lone pairs on the oxygen atom cause water to have a [bent](/wiki/Bent_\\(chemistry\\) "Bent \\(chemistry\\)"), not [linear](/wiki/Linear_molecular_geometry "Linear molecular geometry"), molecular structure,[74] allowing it to be polar. The hydrogen–oxygen')
INFO:answerbot.react:System('Reflect on the received information and plan next steps.')
INFO:answerbot.react:Assistant("The information provided does not give the specific atomic and molecular weights needed to calculate the weight proportion of oxygen in wat